In [ ]:
import os
import pyaudio
#import sounddevice as sd
import wave
import numpy as np
import time
import scipy.io.wavfile as wav
import pyttsx3
import torch
from time import sleep
import speech_recognition as sr
import pandas as pd
import joblib
from tensorflow.keras.preprocessing.sequence import pad_sequences
from speechbrain.inference.speaker import SpeakerRecognition
from pydub import AudioSegment
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")


#app/Server/pretrained_models/spkrec-ecapa-voxceleb
verification = SpeakerRecognition.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb", savedir="app/Server/pretrained_models/spkrec-ecapa-voxceleb")
# --- Model Loading ---
model = joblib.load('./app/Server/anirbanModels/Main/RNN_Multiclass_Subclass.joblib')
tokenizer = joblib.load('./app/Server/anirbanModels/Main/tokenizert.joblib')
max_sequence_length = joblib.load('./app/Server/anirbanModels/Main/max_sequence_length_mainModel.joblib')

model_E = joblib.load('./app/Server/anirbanModels/Edge/RNN_Multiclass_SubclassCategory.joblib')
tokenizer_E = joblib.load('./app/Server/anirbanModels/Edge/tokenizer.joblib')
max_sequence_length_E = joblib.load('./app/Server/anirbanModels/Edge/max_sequence_length.joblib')

model_C = joblib.load('./app/Server/anirbanModels/Cloud/RNN_Multiclass_SubclassCategory.joblib')
tokenizer_C = joblib.load('./app/Server/anirbanModels/Cloud/tokenizer.joblib')
max_sequence_length_C = joblib.load('./app/Server/anirbanModels/Cloud/max_sequence_length.joblib')

model_U = joblib.load('./app/Server/anirbanModels/Update/RNN_Multiclass_SubclassCategory.joblib')
tokenizer_U = joblib.load('./app/Server/anirbanModels/Update/tokenizer.joblib')
max_sequence_length_U = joblib.load('./app/Server/anirbanModels/Update/max_sequence_length.joblib')


# Initialize the speech engine
engine = pyttsx3.init()

def speak(text):
    print(text)
    engine.say(text)
    engine.runAndWait()
def record_audio(file_path, duration=10, audioName="None"):
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 2
    RATE = 48000
    RECORD_SECONDS = duration
    WAVE_OUTPUT_FILENAME = file_path
    audio = pyaudio.PyAudio()
    stream = audio.open(format=FORMAT, channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK)
   
       
   
    if(audioName=="HeyTvs"):
        speak("Please say the wake word 'Hey TVS'")
        sleep(2)
    print("Recording...")
    frames = []
    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)
    print("Finished recording.")
    stream.stop_stream()
    stream.close()
    audio.terminate()
    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(audio.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()



def transcribe_google(audio_path):
    r = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)

    try:
        text = r.recognize_google(audio)
        return text.strip().lower()
    except sr.UnknownValueError:
        return "could not understand audio"
    except sr.RequestError as e:
        return f"could not request results; {e}"


# Predict helper
def predict_class(command, tokenizer, model, max_sequence_length):
    sequence = tokenizer.texts_to_sequences([command])
    padded_sequence = pad_sequences(sequence, maxlen=max_sequence_length, padding='post')
    prediction = model.predict(padded_sequence)
    return np.argmax(prediction)

def Edge_Model(_, command, model, tokenizer, max_sequence_length):
    prediction = predict_class(command, tokenizer, model, max_sequence_length)
    return ["Battery Fuel", "Tires", "Greetings", "Basic"][prediction] if prediction < 4 else "Unknown"

def Cloud_Model(_, command, model, tokenizer, max_sequence_length):
    prediction = predict_class(command, tokenizer, model, max_sequence_length)
    return ["Song Media", "News Notification", "Weather", "Traffic Maps"][prediction] if prediction < 4 else "Unknown"

def Update_Model(_, command, model, tokenizer, max_sequence_length):
    prediction = predict_class(command, tokenizer, model, max_sequence_length)
    return ["Cancel", "Perform", "Check"][prediction] if prediction < 3 else "Unknown"

def Main_model(_, command):
    prediction = predict_class(command, tokenizer, model, max_sequence_length)
    if prediction == 0:
        return ["Edge", Edge_Model(_, command, model_E, tokenizer_E, max_sequence_length_E)]
    elif prediction == 1:
        return ["Cloud", Cloud_Model(_, command, model_C, tokenizer_C, max_sequence_length_C)]
    elif prediction == 2:
        return ["Update", Update_Model(_, command, model_U, tokenizer_U, max_sequence_length_U)]
    elif prediction == 3:
        return ["Miscellaneous", "Temporary"]
    return ["Unknown", "Unknown"]

def sunamdha(command):
    pipeline = joblib.load('./app/Server/Models/ClassSvm.joblib')
    df = pd.DataFrame({'Commands': [command]})
    return pipeline.predict(df['Commands'])[0]

def commandClassify_Sunamdha(class_label_to_predict, text_result):
    df = pd.DataFrame({'Commands': [text_result]})
    if class_label_to_predict in ['Miscellaneous']:
        predicted_subclass = ['Temporary']
    elif class_label_to_predict in ['Not needed']:
        predicted_subclass = ['NC']
    else:
        model_path = f'./app/Server/Models/SubClassModel_{class_label_to_predict}.joblib'
        subClassModel = joblib.load(model_path)
        predicted_subclass = subClassModel.predict(df['Commands'])
    return [class_label_to_predict, predicted_subclass[0]]

def commandClassifier(command):
    subclasscat = "MAINMODEL"
    opLst = Main_model(subclasscat, command)
    subclass_cat = commandClassify_Sunamdha(opLst[0], command)
    return [opLst[0], subclass_cat[1], 0, 0]  # timing skipped for simplicity
def SRSR(username, command_file):
    try:
        dyn_thre = None
        with open("app/Server/dynamic-content/dynamicThreshold.txt", "r") as f:
            for line in f:
                uname, value = line.strip().split(": ")
                if uname == username:
                    dyn_thre = torch.tensor(float(value.replace('tensor([', '').replace('])', '')))
                    break

        if dyn_thre is None:
            print("Dynamic threshold not found for user.")
            return False

        ref_path = None
        with open("app/Server/dynamic-content/users.txt", "r") as f:
            for line in f:
                uname, path = line.strip().split(": ")
                if uname == username:
                    ref_path = path.strip()
                    break

        if not ref_path or not os.path.exists(ref_path):
            print("Reference file not found.")
            return False

        # Step 1 - Direct verification
        ref_path=ref_path.replace("\\", "/")
        score, _ = verification.verify_files(ref_path, command_file)
        score = score.item()
        print(f" Direct score = {score:.4f} | Threshold = {dyn_thre}")

        if score >= dyn_thre:
            return True
        elif score < 0.3:
            print("Score too low (< 0.3). Rejecting.")
            return False
        else:
            # Step 2 - Try wake-word merged verification
            wake_path = f"./app/Server/dynamic-content/heyTVS/{username}_heyTVS.wav"
            if os.path.exists(wake_path):
                user_audio = AudioSegment.from_wav(wake_path)
                cmd_audio = AudioSegment.from_wav(command_file)
                merged = user_audio + cmd_audio
                merged_path = "./app/Server/dynamic-content/unknownVoices/merged_audio_heyTVS.wav"
                merged.export(merged_path, format="wav")

                score2, _ = verification.verify_files(ref_path, merged_path)
                score2 = score2.item()
                print(f" Merged score = {score2:.4f}")

                if score2 >= dyn_thre or score2 >= 0.7:
                    return True
                else:
                    return False
            else:
                print("Wake word audio not found.")
                return False
    except Exception as e:
        print(f"Error in SRSR: {e}")
        return False
def register():
    # Base paths
    base_path = "./app/Server/dynamic-content"
    users_file_path = os.path.join(base_path, "users.txt")
    names_file_path = os.path.join(base_path, "names.txt")
    standard_path = os.path.join(base_path, "final_standard")
    predict_temp_path = os.path.join(base_path, "predictTemp")

    # Wake word paths
    wake_path = os.path.join(base_path, "heyTVS")
    wake_file_path = os.path.join(base_path, "heyTVS.txt")

    # Command paths
    command_folder_path = os.path.join(base_path, "commands")
    command_text_file = os.path.join(base_path, "commands.txt")

    # Threshold log path
    threshold_file_path = os.path.join(base_path, "dynamicThreshold.txt")

    # Get username
    speak("Please say your name")
    username = input("Enter username: ").strip()

    if not username:
        speak("Username cannot be empty")
        return

    # Check if name already exists
    if os.path.exists(names_file_path):
        with open(names_file_path, "r") as file:
            existing_names = [line.strip().lower() for line in file]
        if username.lower() in existing_names:
            speak("Username already exists. Please try a different name.")
            return

    # File paths
    user_audio_path = os.path.join(standard_path, f"{username}.wav")
    user_audio_predict_path = os.path.join(predict_temp_path, f"{username}_predict.wav")
    wake_audio_filename = f"{username}_heyTVS.wav"
    wake_audio_path = os.path.join(wake_path, wake_audio_filename)
    command_audio_filename = f"{username}_command.wav"
    command_audio_path = os.path.join(command_folder_path, command_audio_filename)
    relative_wake_path = f"dynamic-content/heyTVS/{wake_audio_filename}"
    relative_command_path = f"app/Server/dynamic-content/commands/{command_audio_filename}"

    # Record reference and verification audio
    speak("Recording reference audio")
    sleep(1)
    record_audio(user_audio_path)

    speak("Recording verification audio")
    sleep(1)
    record_audio(user_audio_predict_path)

    try:
        score, _ = verification.verify_files(user_audio_path, user_audio_predict_path)
        score = score.item()
        print(user_audio_path, user_audio_predict_path)
        print(f"Verification score: {score:.4f}")

        if score >= 0.7:
            # Save reference and name info
            with open(users_file_path, "a") as ufile:
                
                ufile.write(f"{username}: {user_audio_path}\n")
            with open(names_file_path, "a") as nfile:
                nfile.write(f"{username}\n")

            # Record wake word
            record_audio(wake_audio_path, duration=2, audioName="HeyTvs")
            with open(wake_file_path, "a") as wfile:
                wfile.write(f"{username}: {relative_wake_path}\n")

            # Record and verify standard command
            command_attempts = 0
            max_attempts = 8
            command_verified = False

            while command_attempts < max_attempts and not command_verified:
                speak("Please say the standard command: 'Current weather in Bangalore is very nice?'")
                sleep(1)
                record_audio(command_audio_path, duration=5)

                try:
                    cmd_score, _ = verification.verify_files(user_audio_path, command_audio_path)
                    cmd_score = cmd_score.item()
                    print(f"Command match score: {cmd_score:.4f}")

                    if 0.475 <= cmd_score <= 0.6:
                        with open(command_text_file, "a") as cfile:
                            cfile.write(f"{username}: {relative_command_path}\n")

                        # Save threshold score
                        with open(threshold_file_path, "a") as tfile:
                            tfile.write(f"{username}: tensor([{cmd_score:.4f}])\n")

                        command_verified = True
                        speak(f"Registration successful for {username}")
                        break
                    else:
                        speak("Command does not match. Please try again.")
                        command_attempts += 1

                except Exception as e:
                    speak(f"Error verifying standard command: {str(e)}")
                    command_attempts += 1

            if not command_verified:
                speak("Registration failed due to repeated command mismatch. Cleaning up.")
                cleanup_user_data(username, [
                    user_audio_path,
                    user_audio_predict_path,
                    wake_audio_path,
                    command_audio_path
                ], [
                    users_file_path,
                    names_file_path,
                    wake_file_path,
                    command_text_file,
                    threshold_file_path
                ])

        else:
            speak("Voice match failed. Registration unsuccessful.")
            if os.path.exists(user_audio_path):
                os.remove(user_audio_path)
            if os.path.exists(user_audio_predict_path):
                os.remove(user_audio_predict_path)

    except Exception as e:
        speak(f"Error during registration: {str(e)}")
        if os.path.exists(user_audio_path):
            os.remove(user_audio_path)
        if os.path.exists(user_audio_predict_path):
            os.remove(user_audio_predict_path)

# Helper: Remove user entries and files
def cleanup_user_data(username, file_paths, text_files):
    for file_path in file_paths:
        if os.path.exists(file_path):
            os.remove(file_path)

    def remove_line(file, key):
        if os.path.exists(file):
            with open(file, "r") as f:
                lines = f.readlines()
            with open(file, "w") as f:
                for line in lines:
                    if not line.lower().startswith(key.lower()):
                        f.write(line)

    for txt_file in text_files:
        remove_line(txt_file, username)


def login():
    user_file_path = "./app/Server/dynamic-content/users.txt"
    speak("Recording login audio")
    sleep(1)
    record_audio("./app/Server/dynamic-content/unknownVoices/unknown.wav")

    user_scores = {}
    if not os.path.exists(user_file_path):
        speak("User reference file not found.")
        return

    with open(user_file_path, "r") as file:
        for line in file:
            if ':' in line:
                name, ref_path = line.strip().split(':', 1)
                name, ref_path = name.strip(), ref_path.strip()
                if os.path.exists(ref_path):
                    try:
                        ref_path=ref_path.replace("\\", "/")
                        score, _ = verification.verify_files(ref_path, "./app/Server/dynamic-content/unknownVoices/unknown.wav")
                        x="./app/Server/dynamic-content/unknownVoices/unknown.wav"
                        print(score.item(),ref_path,x)
                        user_scores[name] = score.item()
                        print(f"Score for {name}: {score}")
                    except Exception as e:
                        print(f"Error processing {name}: {e}")
                else:
                    print(f"Reference audio not found for {name}: {ref_path}")

    if user_scores:
        best_match = max(user_scores, key=user_scores.get)
        max_score = user_scores[best_match]

        if max_score >= 0.7:
            speak(f"Welcome {best_match} to OTA Voice Assistant")
            while True:
                speak("Listening for a wake word: 'Hey Bike'")
                record_audio("temp_trigger.wav", duration=3)
                keyword = transcribe_google("temp_trigger.wav")
                print(f"Recognized keyword: {keyword}")

                if "logout" in keyword:
                    speak("Logging out. Goodbye!")
                    break

                elif "hey bike" in keyword:
                    speak("How can I help you?")
                    input_cmd_path = "./app/Server/dynamic-content/inputCmd/inputCmd.wav"
                    record_audio(input_cmd_path, duration=5)
                    command = transcribe_google(input_cmd_path)
                    print(f"User command: {command}")

                    # ✅ Verify speaker before classification
                    if SRSR(best_match, input_cmd_path):
                        # LSTM Prediction
                        anirban_main, anirban_sub, _, _ = commandClassifier(command)
                        print(f"Anirban ➤ Main: {anirban_main}, Sub: {anirban_sub}")

                        # SVM Prediction
                        svm_main = sunamdha(command)
                        svm_main, svm_sub = commandClassify_Sunamdha(anirban_main, command)
                        print(f"Sunamdha ➤ Main: {svm_main}, Sub: {svm_sub}")

                        speak(f"Command recoginized as {anirban_main} with subcategory {svm_sub}")
                    else:
                        speak("Speaker verification failed. Please say 'Hey Bike' again.")
                else:
                    speak("Unrecognized command. Please say 'Logout' to exit the session.")
        else:
            speak("Unauthorized user")
    else:
        speak("No valid users found for verification")

def main():
    print("Select an option:")
    print("1. Register")
    print("2. Login")
    print("3. Exit")
   
    choice = input("Enter your choice (1/2/3): ")

    switch = {
        "1": register,
        "2": login,
        "3":  lambda: (_ for _ in ()).throw(SystemExit)
    }

    action = switch.get(choice)
   
    if action:
        action()
    else:
        print("Invalid choice. Please select 1 or 2.")

if __name__ == "__main__":
    while(1):
        main()


c:\Users\ITinframanage\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\ITinframanage\AppData\Local\Programs\Python\Python312\Lib\site-packages\speechbrain\utils\parameter_transfer.py:234: UserWarning: Requested Pretrainer collection using symlinks on Windows. This might not work; see `LocalStrategy` documentation. Consider unsetting `collect_in` in Pretrainer to avoid symlinking altogether.
  warnings.warn(


Select an option:
1. Register
2. Login
3. Exit
Recording login audio
Recording...
Finished recording.
0.6929559111595154 ./app/Server/dynamic-content/final_standard/maddula.wav ./app/Server/dynamic-content/unknownVoices/unknown.wav
Score for maddula: tensor([0.6930])
0.6509189605712891 ./app/Server/dynamic-content/final_standard/sunamdha.wav ./app/Server/dynamic-content/unknownVoices/unknown.wav
Score for sunamdha: tensor([0.6509])
0.18983744084835052 ./app/Server/dynamic-content/final_standard/pras.wav ./app/Server/dynamic-content/unknownVoices/unknown.wav
Score for pras: tensor([0.1898])
0.2572924494743347 ./app/Server/dynamic-content/final_standard/sandhya.wav ./app/Server/dynamic-content/unknownVoices/unknown.wav
Score for sandhya: tensor([0.2573])
0.042848408222198486 ./app/Server/dynamic-content/final_standard/shivam.wav ./app/Server/dynamic-content/unknownVoices/unknown.wav
Score for shivam: tensor([0.0428])
0.38168904185295105 ./app/Server/dynamic-content/final_standard/ramya.w

SystemExit: 

C:\Users\ITinframanage\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
c=int(input())
for i in range(c, c+5):
    file_path = f"HeyTVSWakeword/sunamdha/{i}.wav"
    record_audio(file_path, duration=2)

In [5]:
import os
import pyaudio
#import sounddevice as sd
import wave
import numpy as np
import time
import scipy.io.wavfile as wav
import pyttsx3
import torch
from time import sleep
import speech_recognition as sr
import pandas as pd
import joblib
from tensorflow.keras.preprocessing.sequence import pad_sequences
from speechbrain.inference.speaker import SpeakerRecognition
from pydub import AudioSegment
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")


#app/Server/pretrained_models/spkrec-ecapa-voxceleb
verification = SpeakerRecognition.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb", savedir="app/Server/pretrained_models/spkrec-ecapa-voxceleb")
# --- Model Loading ---
model = joblib.load('./app/Server/anirbanModels/Main/RNN_Multiclass_Subclass.joblib')
tokenizer = joblib.load('./app/Server/anirbanModels/Main/tokenizert.joblib')
max_sequence_length = joblib.load('./app/Server/anirbanModels/Main/max_sequence_length_mainModel.joblib')

model_E = joblib.load('./app/Server/anirbanModels/Edge/RNN_Multiclass_SubclassCategory.joblib')
tokenizer_E = joblib.load('./app/Server/anirbanModels/Edge/tokenizer.joblib')
max_sequence_length_E = joblib.load('./app/Server/anirbanModels/Edge/max_sequence_length.joblib')

model_C = joblib.load('./app/Server/anirbanModels/Cloud/RNN_Multiclass_SubclassCategory.joblib')
tokenizer_C = joblib.load('./app/Server/anirbanModels/Cloud/tokenizer.joblib')
max_sequence_length_C = joblib.load('./app/Server/anirbanModels/Cloud/max_sequence_length.joblib')

model_U = joblib.load('./app/Server/anirbanModels/Update/RNN_Multiclass_SubclassCategory.joblib')
tokenizer_U = joblib.load('./app/Server/anirbanModels/Update/tokenizer.joblib')
max_sequence_length_U = joblib.load('./app/Server/anirbanModels/Update/max_sequence_length.joblib')


# Initialize the speech engine
engine = pyttsx3.init()

def speak(text):
    print(text)
    engine.say(text)
    engine.runAndWait()
def record_audio(file_path, duration=10, audioName="None"):
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 2
    RATE = 48000
    RECORD_SECONDS = duration
    WAVE_OUTPUT_FILENAME = file_path
    audio = pyaudio.PyAudio()
    stream = audio.open(format=FORMAT, channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK)
   
       
   
    if(audioName=="HeyTvs"):
        speak("Please say the wake word 'Hey TVS'")
        sleep(2)
    print("Recording...")
    frames = []
    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)
    print("Finished recording.")
    stream.stop_stream()
    stream.close()
    audio.terminate()
    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(audio.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()



def transcribe_google(audio_path):
    r = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)

    try:
        text = r.recognize_google(audio)
        return text.strip().lower()
    except sr.UnknownValueError:
        return "could not understand audio"
    except sr.RequestError as e:
        return f"could not request results; {e}"


# Predict helper
def predict_class(command, tokenizer, model, max_sequence_length):
    sequence = tokenizer.texts_to_sequences([command])
    padded_sequence = pad_sequences(sequence, maxlen=max_sequence_length, padding='post')
    prediction = model.predict(padded_sequence)
    return np.argmax(prediction)

def Edge_Model(_, command, model, tokenizer, max_sequence_length):
    prediction = predict_class(command, tokenizer, model, max_sequence_length)
    return ["Battery Fuel", "Tires", "Greetings", "Basic"][prediction] if prediction < 4 else "Unknown"

def Cloud_Model(_, command, model, tokenizer, max_sequence_length):
    prediction = predict_class(command, tokenizer, model, max_sequence_length)
    return ["Song Media", "News Notification", "Weather", "Traffic Maps"][prediction] if prediction < 4 else "Unknown"

def Update_Model(_, command, model, tokenizer, max_sequence_length):
    prediction = predict_class(command, tokenizer, model, max_sequence_length)
    return ["Cancel", "Perform", "Check"][prediction] if prediction < 3 else "Unknown"

def Main_model(_, command):
    prediction = predict_class(command, tokenizer, model, max_sequence_length)
    if prediction == 0:
        return ["Edge", Edge_Model(_, command, model_E, tokenizer_E, max_sequence_length_E)]
    elif prediction == 1:
        return ["Cloud", Cloud_Model(_, command, model_C, tokenizer_C, max_sequence_length_C)]
    elif prediction == 2:
        return ["Update", Update_Model(_, command, model_U, tokenizer_U, max_sequence_length_U)]
    elif prediction == 3:
        return ["Miscellaneous", "Temporary"]
    return ["Unknown", "Unknown"]

def sunamdha(command):
    pipeline = joblib.load('./app/Server/Models/ClassSvm.joblib')
    df = pd.DataFrame({'Commands': [command]})
    return pipeline.predict(df['Commands'])[0]

def commandClassify_Sunamdha(class_label_to_predict, text_result):
    df = pd.DataFrame({'Commands': [text_result]})
    if class_label_to_predict in ['Miscellaneous']:
        predicted_subclass = ['Temporary']
    elif class_label_to_predict in ['Not needed']:
        predicted_subclass = ['NC']
    else:
        model_path = f'./app/Server/Models/SubClassModel_{class_label_to_predict}.joblib'
        subClassModel = joblib.load(model_path)
        predicted_subclass = subClassModel.predict(df['Commands'])
    return [class_label_to_predict, predicted_subclass[0]]

def commandClassifier(command):
    subclasscat = "MAINMODEL"
    opLst = Main_model(subclasscat, command)
    subclass_cat = commandClassify_Sunamdha(opLst[0], command)
    return [opLst[0], subclass_cat[1], 0, 0]  # timing skipped for simplicity
def SRSR(username, command_file):
    try:
        dyn_thre = None
        with open("app/Server/dynamic-content/dynamicThreshold.txt", "r") as f:
            for line in f:
                uname, value = line.strip().split(": ")
                if uname == username:
                    dyn_thre = torch.tensor(float(value.replace('tensor([', '').replace('])', '')))
                    break

        if dyn_thre is None:
            print("Dynamic threshold not found for user.")
            return False

        ref_path = None
        with open("app/Server/dynamic-content/users.txt", "r") as f:
            for line in f:
                uname, path = line.strip().split(": ")
                if uname == username:
                    ref_path = path.strip()
                    break

        if not ref_path or not os.path.exists(ref_path):
            print("Reference file not found.")
            return False

        # Step 1 - Direct verification
        ref_path=ref_path.replace("\\", "/")
        score, _ = verification.verify_files(ref_path, command_file)
        score = score.item()
        print(f" Direct score = {score:.4f} | Threshold = {dyn_thre}")

        if score >= dyn_thre:
            return True
        elif score < 0.3:
            print("Score too low (< 0.3). Rejecting.")
            return False
        else:
            # Step 2 - Try wake-word merged verification
            wake_path = f"./app/Server/dynamic-content/heyTVS/{username}_heyTVS.wav"
            if os.path.exists(wake_path):
                user_audio = AudioSegment.from_wav(wake_path)
                cmd_audio = AudioSegment.from_wav(command_file)
                merged = user_audio + cmd_audio
                merged_path = "./app/Server/dynamic-content/unknownVoices/merged_audio_heyTVS.wav"
                merged.export(merged_path, format="wav")

                score2, _ = verification.verify_files(ref_path, merged_path)
                score2 = score2.item()
                print(f" Merged score = {score2:.4f}")

                if score2 >= dyn_thre or score2 >= 0.7:
                    return True
                else:
                    return False
            else:
                print("Wake word audio not found.")
                return False
    except Exception as e:
        print(f"Error in SRSR: {e}")
        return False
def register():
    base_path = "./app/Server/dynamic-content"
    users_file_path = os.path.join(base_path, "users.txt")
    names_file_path = os.path.join(base_path, "names.txt")
    standard_path = os.path.join(base_path, "final_standard")
    predict_temp_path = os.path.join(base_path, "predictTemp")

    wake_path = os.path.join(base_path, "heyTVS")
    wake_file_path = os.path.join(base_path, "heyTVS.txt")

    command_folder_path = os.path.join(base_path, "commands")
    command_text_file = os.path.join(base_path, "commands.txt")
    threshold_file_path = os.path.join(base_path, "dynamicThreshold.txt")

    speak("Please say your name")
    username = input("Enter username: ").strip()

    if not username:
        speak("Username cannot be empty")
        return

    if os.path.exists(names_file_path):
        with open(names_file_path, "r") as file:
            existing_names = [line.strip().lower() for line in file]
        if username.lower() in existing_names:
            speak("Username already exists. Please try a different name.")
            return

    user_audio_path = os.path.join(standard_path, f"{username}.wav")
    user_audio_predict_path = os.path.join(predict_temp_path, f"{username}_predict.wav")
    wake_audio_filename = f"{username}_heyTVS.wav"
    wake_audio_path = os.path.join(wake_path, wake_audio_filename)
    command_audio_filename = f"{username}_command.wav"
    command_audio_path = os.path.join(command_folder_path, command_audio_filename)
    relative_wake_path = f"dynamic-content/heyTVS/{wake_audio_filename}"
    relative_command_path = f"app/Server/dynamic-content/commands/{command_audio_filename}"

    speak("Recording reference audio")
    sleep(1)
    record_audio(user_audio_path)

    speak("Recording verification audio")
    sleep(1)
    record_audio(user_audio_predict_path)

    try:
        score, _ = verification.verify_files(user_audio_path, user_audio_predict_path)
        score = score.item()
        print(user_audio_path, user_audio_predict_path)
        print(f"Verification score: {score:.4f}")

        if score >= 0.7:
            with open(users_file_path, "a") as ufile:
                ufile.write(f"{username}: {user_audio_path}\n")
            with open(names_file_path, "a") as nfile:
                nfile.write(f"{username}\n")

            record_audio(wake_audio_path, duration=2, audioName="HeyTvs")
            with open(wake_file_path, "a") as wfile:
                wfile.write(f"{username}: {relative_wake_path}\n")

            # Command recording phase
            command_attempts = 0
            max_attempts = 8
            command_verified = False
            cmd_score = 0.0  # initialize to avoid reference error

            # Define versions of the command
            very_short_command = "Weather in Bangalore"
            short_command = "Current weather in Bangalore"
            default_command = "Current weather in Bangalore is very nice?"
            long_command = "Current weather in Bangalore is very nice today and I love it"
            very_long_command = "Current weather in Bangalore is very nice today, I love the temperature and climate so much"

            while command_attempts < max_attempts and not command_verified:
                # Choose prompt based on previous score
                if command_attempts == 0:
                    current_prompt = default_command
                else:
                    if cmd_score < 0.45:
                        current_prompt = very_long_command
                    elif cmd_score < 0.475:
                        current_prompt = long_command
                    elif cmd_score > 0.65:
                        current_prompt = very_short_command
                    elif cmd_score > 0.6:
                        current_prompt = short_command
                    else:
                        current_prompt = default_command

                speak(f"Please say the command: '{current_prompt}'")
                sleep(1)
                record_audio(command_audio_path, duration=5)

                try:
                    cmd_score, _ = verification.verify_files(user_audio_path, command_audio_path)
                    cmd_score = cmd_score.item()
                    print(f"Command match score: {cmd_score:.4f}")

                    if 0.475 <= cmd_score <= 0.6:
                        with open(command_text_file, "a") as cfile:
                            cfile.write(f"{username}: {relative_command_path}\n")
                        with open(threshold_file_path, "a") as tfile:
                            tfile.write(f"{username}: tensor([{cmd_score:.4f}])\n")

                        command_verified = True
                        speak(f"Registration successful for {username}")
                        break
                    else:
                        speak("Command does not match. Retrying with adjusted prompt...")
                        command_attempts += 1

                except Exception as e:
                    speak(f"Error verifying standard command: {str(e)}")
                    command_attempts += 1

            if not command_verified:
                speak("Registration failed due to repeated command mismatch. Cleaning up.")
                cleanup_user_data(username, [
                    user_audio_path,
                    user_audio_predict_path,
                    wake_audio_path,
                    command_audio_path
                ], [
                    users_file_path,
                    names_file_path,
                    wake_file_path,
                    command_text_file,
                    threshold_file_path
                ])

        else:
            speak("Voice match failed. Registration unsuccessful.")
            if os.path.exists(user_audio_path):
                os.remove(user_audio_path)
            if os.path.exists(user_audio_predict_path):
                os.remove(user_audio_predict_path)

    except Exception as e:
        speak(f"Error during registration: {str(e)}")
        if os.path.exists(user_audio_path):
            os.remove(user_audio_path)
        if os.path.exists(user_audio_predict_path):
            os.remove(user_audio_predict_path)

# Helper: Remove user entries and files
def cleanup_user_data(username, file_paths, text_files):
    for file_path in file_paths:
        if os.path.exists(file_path):
            os.remove(file_path)

    def remove_line(file, key):
        if os.path.exists(file):
            with open(file, "r") as f:
                lines = f.readlines()
            with open(file, "w") as f:
                for line in lines:
                    if not line.lower().startswith(key.lower()):
                        f.write(line)

    for txt_file in text_files:
        remove_line(txt_file, username)


def login():
    user_file_path = "./app/Server/dynamic-content/users.txt"
    speak("Recording login audio")
    sleep(1)
    record_audio("./app/Server/dynamic-content/unknownVoices/unknown.wav")

    user_scores = {}
    if not os.path.exists(user_file_path):
        speak("User reference file not found.")
        return

    with open(user_file_path, "r") as file:
        for line in file:
            if ':' in line:
                name, ref_path = line.strip().split(':', 1)
                name, ref_path = name.strip(), ref_path.strip()
                if os.path.exists(ref_path):
                    try:
                        ref_path=ref_path.replace("\\", "/")
                        score, _ = verification.verify_files(ref_path, "./app/Server/dynamic-content/unknownVoices/unknown.wav")
                        x="./app/Server/dynamic-content/unknownVoices/unknown.wav"
                        print(score.item(),ref_path,x)
                        user_scores[name] = score.item()
                        print(f"Score for {name}: {score}")
                    except Exception as e:
                        print(f"Error processing {name}: {e}")
                else:
                    print(f"Reference audio not found for {name}: {ref_path}")

    if user_scores:
        best_match = max(user_scores, key=user_scores.get)
        max_score = user_scores[best_match]

        if max_score >= 0.7:
            speak(f"Welcome {best_match} to OTA Voice Assistant")
            while True:
                speak("Listening for a wake word: 'Hey Bike'")
                record_audio("temp_trigger.wav", duration=3)
                keyword = transcribe_google("temp_trigger.wav")
                print(f"Recognized keyword: {keyword}")

                if "logout" in keyword:
                    speak("Logging out. Goodbye!")
                    break

                elif "hey bike" in keyword:
                    speak("How can I help you?")
                    input_cmd_path = "./app/Server/dynamic-content/inputCmd/inputCmd.wav"
                    record_audio(input_cmd_path, duration=5)
                    command = transcribe_google(input_cmd_path)
                    print(f"User command: {command}")

                    # ✅ Verify speaker before classification
                    if SRSR(best_match, input_cmd_path):
                        # LSTM Prediction
                        anirban_main, anirban_sub, _, _ = commandClassifier(command)
                        print(f"Anirban ➤ Main: {anirban_main}, Sub: {anirban_sub}")

                        # SVM Prediction
                        svm_main = sunamdha(command)
                        svm_main, svm_sub = commandClassify_Sunamdha(anirban_main, command)
                        print(f"Sunamdha ➤ Main: {svm_main}, Sub: {svm_sub}")

                        speak(f"Command recoginized as {anirban_main} with subcategory {svm_sub}")
                    else:
                        speak("Speaker verification failed. Please say 'Hey Bike' again.")
                else:
                    speak("Unrecognized command. Please say 'Logout' to exit the session.")
        else:
            speak("Unauthorized user")
    else:
        speak("No valid users found for verification")

def main():
    print("Select an option:")
    print("1. Register")
    print("2. Login")
    print("3. Exit")
   
    choice = input("Enter your choice (1/2/3): ")

    switch = {
        "1": register,
        "2": login,
        "3":  lambda: (_ for _ in ()).throw(SystemExit)
    }

    action = switch.get(choice)
   
    if action:
        action()
    else:
        print("Invalid choice. Please select 1 or 2.")

if __name__ == "__main__":
    while(1):
        main()


c:\Users\ITinframanage\AppData\Local\Programs\Python\Python312\Lib\site-packages\speechbrain\utils\parameter_transfer.py:234: UserWarning: Requested Pretrainer collection using symlinks on Windows. This might not work; see `LocalStrategy` documentation. Consider unsetting `collect_in` in Pretrainer to avoid symlinking altogether.
  warnings.warn(


Select an option:
1. Register
2. Login
3. Exit
Recording login audio
Recording...
Finished recording.
0.6872966289520264 ./app/Server/dynamic-content/final_standard/maddula.wav ./app/Server/dynamic-content/unknownVoices/unknown.wav
Score for maddula: tensor([0.6873])
0.6456663012504578 ./app/Server/dynamic-content/final_standard/sunamdha.wav ./app/Server/dynamic-content/unknownVoices/unknown.wav
Score for sunamdha: tensor([0.6457])
0.19018082320690155 ./app/Server/dynamic-content/final_standard/pras.wav ./app/Server/dynamic-content/unknownVoices/unknown.wav
Score for pras: tensor([0.1902])
0.2291005253791809 ./app/Server/dynamic-content/final_standard/sandhya.wav ./app/Server/dynamic-content/unknownVoices/unknown.wav
Score for sandhya: tensor([0.2291])
0.10770078003406525 ./app/Server/dynamic-content/final_standard/shivam.wav ./app/Server/dynamic-content/unknownVoices/unknown.wav
Score for shivam: tensor([0.1077])
0.38387563824653625 ./app/Server/dynamic-content/final_standard/ramya.wa

SystemExit: 

C:\Users\ITinframanage\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
